## Agentic RAG
이번 챕터에서는 문서 검색을 통해 최신 정보에 접근하여 검색 결과를 가지고 답변을 생성하는 에이전트를 만들어 보겠습니다.

질문에 따라 문서를 검색하여 답변하거나, 인터넷 검색 도구를 활용하여 답변하는 에이전트를 만들어 보겠습니다.

**참고**

- RAG 를 수행하되, Agent 를 활용하여 RAG 를 수행한다면 이를 `Agentic RAG` 라고 부릅니다.

### 도구(Tools)
Agent 가 활용할 도구를 정의하여 Agent 가 추론(reasoning)을 수행할 때 활용하도록 만들 수 있습니다.

Tavily Search 는 그 중 대표적인 `검색 도구` 입니다. 검색을 통해 최신 정보에 접근하여 검색 결과를 가지고 답변을 생성할 수 있습니다. 도구는 이처럼 검색 도구 뿐만아니라 Python 코드를 실행할 수 있는 도구, 직접 정의한 함수를 실행하는 도구 등 다양한 종류와 방법론을 제공합니다.

### `웹 검색도구: Tavily Search`
LangChain에는 Tavily 검색 엔진을 도구로 쉽게 사용할 수 있는 내장 도구가 있습니다.

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from langchain_teddynote import logging
logging.langsmith("CH15-Agents")

LangSmith 추적을 시작합니다.
[프로젝트명]
CH15-Agents


In [3]:
from langchain_community.tools.tavily_search import TavilySearchResults
search = TavilySearchResults(k=6)

In [4]:
response = search.invoke("판교 카카오 프렌즈샵 아지트점의 전화번호는 무엇인가요?")

In [5]:
print(response)

[{'url': 'https://blog.naver.com/PostView.naver?blogId=amour1115&logNo=223386213560', 'content': '카카오 프렌즈 판교 아지트점. . 연중무휴 12:00-21:00. 토, 일 12:00-20:00. . 경기 성남시 분당구 판교역로 166. 카카오 판교 아지트 1층 카카오 프렌즈. 031-301-7225. 존재하지 않는 이미지입니다.'}, {'url': 'https://blog.naver.com/PostView.naver?blogId=ifnotnow_thenwhen&logNo=223203793513&noTrackingCode=true', 'content': '11. 17:00. 판교 카카오아지트는 판교역 4번 출구에 있습니다. 층을 걸쳐 번쩍번쩍 전광판에.. 나 이런데 다닌다.. 하면 간지나겠더라고용 ㅎㅎ. 어쩜 이렇게 질리지도 않게 캐릭터들을 잘 만드는지.. 분명 천재들이 틀림없습니다!! ㅎㅎ. 판교현대백화점에서도 ...'}, {'url': 'https://kr.trip.com/moments/detail/seongnam-121856-121058702/', 'content': '사진찍으면서 실내데이트 즐기고 싶다면 주목📢 판교역에서 꼭 가봐야할 아기자기 귀여운 핫플레이스 카카오 프렌즈 캐릭터들의 포토존이 가득한 대형 카카오 프렌즈를 판교점은 정말 특별해요! 📍카카오프렌즈 판교 아지트점 ️경기도 성남시 분당구 판교역로 166 카카오판교아지트 다른 카카오 ...'}, {'url': 'https://m.blog.naver.com/ifnotnow_thenwhen/223203793513', 'content': '이웃추가. 카카오프렌즈샵 판교아지트. 존재하지 않는 이미지입니다. 정말 국민 캐릭터라고 해도 크게 반발할 사람 없는. #카카오프렌즈. 남편 친구가 판교 카카오아지트에서 일한다고 들었는데. 짱 좋다길래 한번 구경 가보고 싶었어요 ♥. 존재하지 않는 ...'}, {'url': 'https://

In [6]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain.document_loaders import PyPDFLoader

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

loader =PyPDFLoader("data/SPRI_AI_Brief_2023년12월호_F.pdf")
docs = loader.load()
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=50
)

documents = text_splitter.split_documents(docs)

vector = FAISS.from_documents(documents, embeddings)

retriever = vector.as_retriever()

In [7]:
from langchain.tools.retriever import create_retriever_tool

retriever_tool = create_retriever_tool(
    retriever,
    name="pdf_search",
    description="use this tool to search information from the PDF document" # 도구에 대한 자세한 설명을 기입해야 한다.
)

In [8]:
# Agent가 사용할 도구 목록 정의
tools = [search, retriever_tool]

In [9]:
# Agent 생성
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

llm = ChatOpenAI(model="gpt-4o-mini")

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. "
            "Make sure to use the `pdf_search` tool for searching information from the PDF document. "
            "If you can't find the information from the PDF document, use the `search` tool for searching information from the web.",
        ),
        ("placeholder", "{chat_history}"),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}")
    ]
)

In [10]:
# Tool Calling Agent 를 생성
from langchain.agents import create_tool_calling_agent

agent = create_tool_calling_agent(llm, tools, prompt)

In [11]:
# Agent를 실행하기 위한 AgentExecutor
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

### 에이전트 실행하기
이제 몇 가지 질의에 대해 에이전트를 실행할 수 있습니다!

현재 이러한 모든 질의는 **상태(Stateless)** 가 없는 질의입니다(이전 상호작용을 기억하지 않습니다).

`agent_executor` 객체의 `invoke` 메소드는 딕셔너리 형태의 인자를 받아 처리합니다. 이 예제에서는 `input` 키에 `hi!` 값을 할당한 딕셔너리를 인자로 전달하고 있습니다. 이는 일반적으로 AI 에이전트, 함수 실행기, 또는 명령 처리기 등의 객체에서 입력을 처리하기 위해 사용됩니다.

In [12]:
result = agent_executor.invoke(
    {"input": "2024년 프로야구 플레이오프 진출한 5개 팀을 검색하여 알려주세요."}
)



> Entering new AgentExecutor chain...

Invoking: `tavily_search_results_json` with `{'query': '2024 프로야구 플레이오프 진출 팀'}`


[{'url': 'https://realjace.tistory.com/entry/2024-프로야구-포스트시즌-플레이오프-한국시리즈-일정-총정리', 'content': "2024 프로야구 포스트시즌 플레이오프 한국시리즈 일정 총정리 2024 프로야구 포스트시즌 플레이오프 한국시리즈 일정 총정리 2024 프로야구 포스트시즌 경기방식 2024 프로야구 포스트시즌 일정 2024 프로야구 포스트시즌 경기방식 와일드카드 결정전에서 승리한 팀과 정규시즌 3위 팀이 맞붙습니다. 준플레이오프에서 승리한 팀과 정규시즌 2위 팀이 경기를 펼치게 됩니다. 플레이오프 승리팀과 정규시즌 1위 팀이 7전 4선 승제로 경기를 펼치게 됩니다. 2024 프로야구 포스트시즌 진출팀 특히 4위부터 6위까지 승차가 거의 없기 때문에 마지막까지 흥미진진할 것으로 보이는데요, 각 팀이 144경기를 소화해야 하며 9월 28일까지 정규시즌이 마무리될 것으로 보입니다. 2024 프로야구 포스트시즌 일정 플레이오프 1차전 플레이오프 2차전 플레이오프 3차전 플레이오프 4차전 '스포츠&게임 정보' 카테고리의 다른 글 2024 프로야구 포스트시즌 티켓 예매방법\xa0\xa0(3) 2024 프로야구 올스타전 투표결과 경기일정 티켓 예매방법 가격 총정리\xa0\xa0(0) 2024 프로야구 포스트시즌 티켓 예매방법 티스토리 홈 이동"}, {'url': 'https://aldalsuin.tistory.com/entry/2024-프로야구-플레이오프-9개-구단의-치열한-우승-경쟁', 'content': '2024 프로야구 플레이오프에서는 각 팀의 전략이 승부의 관건이 될 것으로 보입니다. 플레이오프에서는 한 번의 실수가 경기의 흐름을 바꿀 수 있기 때문에, 수비력이 뛰어난 선수들의 기용이 늘어날 것으로 

In [13]:
from langchain_teddynote.messages import AgentStreamParser

agent_stream_parser = AgentStreamParser()

In [14]:
# 질의에 대한 답변을 스트리밍으로 출력 요청
result = agent_executor.stream(
    {"input": "2024년 프로야구 플레이오프 진출한 5개 팀을 검색하여 알려주세요."}
)

for step in result:
    # 중간 단계를 parser 를 사용하여 단계별로 출력
    agent_stream_parser.process_agent_steps(step)



> Entering new None chain...
[도구 호출]
Tool: tavily_search_results_json
query: 2024년 프로야구 플레이오프 진출 팀
Log: 
Invoking: `tavily_search_results_json` with `{'query': '2024년 프로야구 플레이오프 진출 팀'}`




Invoking: `tavily_search_results_json` with `{'query': '2024년 프로야구 플레이오프 진출 팀'}`


[{'url': 'https://realjace.tistory.com/entry/2024-프로야구-포스트시즌-플레이오프-한국시리즈-일정-총정리', 'content': "2024 프로야구 포스트시즌 플레이오프 한국시리즈 일정 총정리 2024 프로야구 포스트시즌 플레이오프 한국시리즈 일정 총정리 2024 프로야구 포스트시즌 경기방식 2024 프로야구 포스트시즌 일정 2024 프로야구 포스트시즌 경기방식 와일드카드 결정전에서 승리한 팀과 정규시즌 3위 팀이 맞붙습니다. 준플레이오프에서 승리한 팀과 정규시즌 2위 팀이 경기를 펼치게 됩니다. 플레이오프 승리팀과 정규시즌 1위 팀이 7전 4선 승제로 경기를 펼치게 됩니다. 2024 프로야구 포스트시즌 진출팀 특히 4위부터 6위까지 승차가 거의 없기 때문에 마지막까지 흥미진진할 것으로 보이는데요, 각 팀이 144경기를 소화해야 하며 9월 28일까지 정규시즌이 마무리될 것으로 보입니다. 2024 프로야구 포스트시즌 일정 플레이오프 1차전 플레이오프 2차전 플레이오프 3차전 플레이오프 4차전 '스포츠&게임 정보' 카테고리의 다른 글 2024 프로야구 포스트시즌 티켓 예매방법\xa0\xa0(3) 2024 프로야구 올스타전 투표결과 경기일정 티켓 예매방법 가격 총정리\xa0\xa0(0) 2024 프로야구 포스트시즌 티켓 예매방법 티스토리 홈 이동"}, {'url': 'https://m.blog.naver.com/hoga1201/2235711178

In [15]:
response = result = agent_executor.invoke(
    {"input": "삼성전자가 자체 개발한 생성형 AI 관련된 정보를 문서에서 찾아주세요."}
)
print(response["output"])



> Entering new AgentExecutor chain...

Invoking: `pdf_search` with `{'query': '삼성전자 생성형 AI'}`


SPRi AI Brief |  
2023-12 월호
10삼성전자 , 자체 개발 생성 AI ‘삼성 가우스 ’ 공개
n삼성전자가 온디바이스에서 작동 가능하며 언어, 코드, 이미지의 3개 모델로 구성된 자체 개발 생성 
AI 모델 ‘삼성 가우스 ’를 공개
n삼성전자는 삼성 가우스를 다양한 제품에 단계적으로 탑재할 계획으로 , 온디바이스 작동이 가능한 
삼성 가우스는 외부로 사용자 정보가 유출될 위험이 없다는 장점을 보유KEY Contents
£언어, 코드, 이미지의 3개 모델로 구성된 삼성 가우스 , 온디바이스 작동 지원
n삼성전자가 2023 년 11월 8일 열린 ‘삼성 AI 포럼 2023’ 행사에서 자체 개발한 생성 AI 모델 
‘삼성 가우스 ’를 최초 공개
∙정규분포 이론을 정립한 천재 수학자 가우스 (Gauss) 의 이름을 본뜬 삼성 가우스는 다양한 상황에 
최적화된 크기의 모델 선택이 가능
∙삼성 가우스는 라이선스나 개인정보를 침해하지 않는 안전한 데이터를 통해 학습되었으며 , 
온디바이스에서 작동하도록 설계되어 외부로 사용자의 정보가 유출되지 않는 장점을 보유
∙삼성전자는 삼성 가우스를 활용한 온디바이스 AI 기술도 소개했으며 , 생성 AI 모델을 다양한 제품에 
단계적으로 탑재할 계획
n삼성 가우스는 △텍스트를 생성하는 언어모델 △코드를 생성하는 코드 모델 △이미지를 생성하는  
이미지 모델의 3개 모델로 구성
∙언어 모델은 클라우드와 온디바이스 대상 다양한 모델로 구성되며 , 메일 작성, 문서 요약, 번역 업무의 
처리를 지원
∙코드 모델 기반의 AI 코딩 어시스턴트 ‘코드아이 (code.i)’ 는 대화형 인터페이스로 서비스를 제공하며 
사내 소프트웨어 개발에 최적화
∙이미지 모델은 창의적인 이미지를 생성하고 기존 이미지를 원하는 대로 바꿀 수 있도록 지원하며 
저해상도 이미지의 

In [16]:
# 질의에 대한 답변을 스트리밍으로 출력 요청
result = agent_executor.stream(
    {"input": "삼성전자가 자체 개발한 생성형 AI 관련된 정보를 문서에서 찾아주세요."}
)

for step in result:
    # 중간 단계를 parser 를 사용하여 단계별로 출력
    agent_stream_parser.process_agent_steps(step)



> Entering new None chain...
[도구 호출]
Tool: pdf_search
query: 삼성전자 자체 개발 생성형 AI
Log: 
Invoking: `pdf_search` with `{'query': '삼성전자 자체 개발 생성형 AI'}`




Invoking: `pdf_search` with `{'query': '삼성전자 자체 개발 생성형 AI'}`


SPRi AI Brief |  
2023-12 월호
10삼성전자 , 자체 개발 생성 AI ‘삼성 가우스 ’ 공개
n삼성전자가 온디바이스에서 작동 가능하며 언어, 코드, 이미지의 3개 모델로 구성된 자체 개발 생성 
AI 모델 ‘삼성 가우스 ’를 공개
n삼성전자는 삼성 가우스를 다양한 제품에 단계적으로 탑재할 계획으로 , 온디바이스 작동이 가능한 
삼성 가우스는 외부로 사용자 정보가 유출될 위험이 없다는 장점을 보유KEY Contents
£언어, 코드, 이미지의 3개 모델로 구성된 삼성 가우스 , 온디바이스 작동 지원
n삼성전자가 2023 년 11월 8일 열린 ‘삼성 AI 포럼 2023’ 행사에서 자체 개발한 생성 AI 모델 
‘삼성 가우스 ’를 최초 공개
∙정규분포 이론을 정립한 천재 수학자 가우스 (Gauss) 의 이름을 본뜬 삼성 가우스는 다양한 상황에 
최적화된 크기의 모델 선택이 가능
∙삼성 가우스는 라이선스나 개인정보를 침해하지 않는 안전한 데이터를 통해 학습되었으며 , 
온디바이스에서 작동하도록 설계되어 외부로 사용자의 정보가 유출되지 않는 장점을 보유
∙삼성전자는 삼성 가우스를 활용한 온디바이스 AI 기술도 소개했으며 , 생성 AI 모델을 다양한 제품에 
단계적으로 탑재할 계획
n삼성 가우스는 △텍스트를 생성하는 언어모델 △코드를 생성하는 코드 모델 △이미지를 생성하는  
이미지 모델의 3개 모델로 구성
∙언어 모델은 클라우드와 온디바이스 대상 다양한 모델로 구성되며 , 메일 작성, 문서 요약, 번역 업무의 
처리를 지원
∙코드 모델 기반의 AI 코딩 어시스턴트 ‘코드아

### 이전 대화내용 기억하는 Agent
이전의 대화내용을 기억하기 위해서는 `RunnableWithMessageHistory` 를 사용하여 `AgentExecutor` 를 감싸줍니다.

`RunnableWithMessageHistory` 에 대한 자세한 내용은 아래 링크를 참고해 주세요.

**참고**

- [RunnableWithMessageHistory](https://wikidocs.net/254682)

In [17]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

# session_id를 저장할 딕셔너리
store = {}

def get_session_history(session_ids):
    if session_ids not in store:
        store[session_ids] = ChatMessageHistory()
    return store[session_ids] # 해당 세션 ID에 대한 세션 기록 반환

In [18]:
agent_with_chat_history = RunnableWithMessageHistory(
    agent_executor,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history"
)

In [19]:
response = agent_with_chat_history.invoke(
    {"input": "삼성전자가 개발한 생성형 AI관련된 정보에 대해서 알려주세요."},
    config={"configurable": {"session_id": "abc123"}}
)
print(response["output"])



> Entering new AgentExecutor chain...

Invoking: `tavily_search_results_json` with `{'query': '삼성전자 생성형 AI'}`


[{'url': 'https://www.yna.co.kr/view/AKR20231108041400003', 'content': "삼성전자[005930]가 자체 개발한 생성형 인공지능(AI) 모델 '삼성 가우스'(Samsung Gauss)가 처음 공개됐다. 삼성전자는 삼성 가우스를 활용해 임직원의 업무 생산성을 높이는 한편, 생성형 AI 모델을 단계적으로 제품에 탑재해 새로운 사용자 경험을 제공한다는 계획이다."}, {'url': 'https://news.samsung.com/kr/삼성전자-삼성-ai-포럼서-자체-개발-생성형-ai-삼성-가', 'content': "삼성전자 자체 개발 생성형 AI 모델 '삼성 가우스' 최초 공개. 이번 포럼에서는 삼성리서치에서 개발한 생성형 AI 모델 '삼성 가우스 (Samsung Gauss)'가 처음으로 공개되어 많은 관심을 받았다. 삼성전자는 '삼성 가우스'를 활용해 회사 내 업무 혁신을 ..."}, {'url': 'https://zdnet.co.kr/view/?no=20231108081251', 'content': "삼성전자가 자체 개발한 생성형 AI 모델 '삼성 가우스 (Samsung Gauss)'를 최초로 공개했다. 삼성전자는 가우스를 활용해 회사 내 업무 혁신을 추진하고 ..."}, {'url': 'https://www.hankyung.com/article/202311081442g', 'content': "사진=삼성전자 제공. 삼성전자가 자체 개발한 생성형 인공지능 (AI) 모델 '삼성 가우스' (Samsung Gauss)가 최초로 공개했다. 삼성전자는 삼성 가우스를 ..."}, {'url': 'https://namu.wiki/w/삼성+가우스', 'content': '삼성전자 뉴스룸. 삼성 가우스 (Samsung Gau

In [20]:
response = agent_with_chat_history.stream(
    {"input": "삼성전자가 개발한 생성형 AI관련된 정보에 대해서 알려주세요."},
    config={"configurable": {"session_id": "abc123"}}
)
for step in response:
    agent_stream_parser.process_agent_steps(step)



> Entering new None chain...
[도구 호출]
Tool: tavily_search_results_json
query: 삼성전자 생성형 AI
Log: 
Invoking: `tavily_search_results_json` with `{'query': '삼성전자 생성형 AI'}`




Invoking: `tavily_search_results_json` with `{'query': '삼성전자 생성형 AI'}`


[{'url': 'https://www.yna.co.kr/view/AKR20231108041400003', 'content': "삼성전자[005930]가 자체 개발한 생성형 인공지능(AI) 모델 '삼성 가우스'(Samsung Gauss)가 처음 공개됐다. 삼성전자는 삼성 가우스를 활용해 임직원의 업무 생산성을 높이는 한편, 생성형 AI 모델을 단계적으로 제품에 탑재해 새로운 사용자 경험을 제공한다는 계획이다."}, {'url': 'https://news.samsung.com/kr/삼성전자-삼성-ai-포럼서-자체-개발-생성형-ai-삼성-가', 'content': "삼성전자 자체 개발 생성형 AI 모델 '삼성 가우스' 최초 공개. 이번 포럼에서는 삼성리서치에서 개발한 생성형 AI 모델 '삼성 가우스 (Samsung Gauss)'가 처음으로 공개되어 많은 관심을 받았다. 삼성전자는 '삼성 가우스'를 활용해 회사 내 업무 혁신을 ..."}, {'url': 'https://zdnet.co.kr/view/?no=20231108081251', 'content': "삼성전자가 자체 개발한 생성형 AI 모델 '삼성 가우스 (Samsung Gauss)'를 최초로 공개했다. 삼성전자는 가우스를 활용해 회사 내 업무 혁신을 추진하고 ..."}, {'url': 'https://www.hankyung.com/article/202311081442g', 'content': "사진=삼성전자 제공. 삼성전자가 자체 개발한 생성형 인공지능 (AI) 모델 '삼성 가

In [21]:
response = agent_with_chat_history.stream(
    {"input": "이전의 답변을 영어로 번역해 주세요."},
    # session_id 설정
    config={"configurable": {"session_id": "abc123"}},
)

# 출력 확인
for step in response:
    agent_stream_parser.process_agent_steps(step)



> Entering new None chain...
Samsung Electronics recently unveiled a generative AI model called "Samsung Gauss." This model was developed by Samsung Research and is set to be gradually integrated into its products to enhance employee productivity and provide new user experiences.

"Samsung Gauss" is a self-developed generative AI designed to drive various business innovations. Samsung Electronics plans to utilize this model to maximize efficiency within the company and offer innovative services to customers.

For more details, you can check the following links:
- [Yonhap News](https://www.yna.co.kr/view/AKR20231108041400003)
- [Samsung Newsroom](https://news.samsung.com/kr/삼성전자-삼성-ai-포럼서-자체-개발-생성형-ai-삼성-가)
- [ZDNet](https://zdnet.co.kr/view/?no=20231108081251)
- [Hankyung](https://www.hankyung.com/article/202311081442g)
- [Newsis](https://www.newsis.com/view/NISX20231108_0002513194)

> Finished chain.
[최종 답변]
Samsung Electronics recently unveiled a generative AI model called "Samsung

In [22]:
# Agent 템플릿

In [23]:
from langchain_community.document_loaders import PDFPlumberLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_community.vectorstores import FAISS
from langchain.tools.retriever import create_retriever_tool

# 1. retriever_tool 만들기

loader = PDFPlumberLoader("./data/SPRI_AI_Brief_2023년12월호_F.pdf")
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=50
)

documents = text_splitter.split_documents(docs)

embeddings = OpenAIEmbeddings(model='text-embedding-3-small')

vectorstore = FAISS.from_documents(documents, embeddings)

retriever = vectorstore.as_retriever()

retriever_tool = create_retriever_tool(
    retriever,
    name="pdf_search",
    description="use this tool to search information from the PDF document",  # 도구에 대한 설명을 자세히 기입해야 합니다!!
)

In [24]:
# 2. 검색 도구 만들기
from langchain.tools.tavily_search import TavilySearchResults
search = TavilySearchResults(k=6)

In [25]:
# 3. tools 정의하기
tools = [search, retriever_tool]

In [26]:
# 4. LLM, Prompt 저의하기
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. "
            "Make sure to use the `pdf_search` tool for searching information from the PDF document. "
            "If you can't find the information from the PDF document, use the `search` tool for searching information from the web.",
        ),
        ("placeholder", "{chat_history}"),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}"),
    ]
)


In [27]:
# 5. Agent 정의하기
from langchain.agents import create_tool_calling_agent
agent = create_tool_calling_agent(llm, tools, prompt)

In [28]:
# 6. AgentExecutor 정의하기
from langchain.agents import AgentExecutor
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [29]:
# 7. 대화내용을 기억하는 Memory 정의하기

from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables import RunnableWithMessageHistory

store = {}

def get_session_history(session_ids):
    if session_ids not in store:
        store["session_ids"] = ChatMessageHistory()
    return store["session_ids"]

agent_with_chat_history = RunnableWithMessageHistory(
    agent_executor,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history"
)


In [30]:
# 8. Agent 실행하기
response = agent_with_chat_history.invoke({"input": "구글이 앤스로픽에 투자한 금액을 문서에서 찾아줘."},
                                          config={"configurable": {"session_id" : "abc123"}})
print(response["output"])



> Entering new AgentExecutor chain...

Invoking: `pdf_search` with `{'query': '구글 앤스로픽 투자 금액'}`


1. 정책/법제 2. 기업/산업 3. 기술/연구 4. 인력/교육
구글, 앤스로픽에 20억 달러 투자로 생성 AI 협력 강화
KEY Contents
n 구글이 앤스로픽에 최대 20억 달러 투자에 합의하고 5억 달러를 우선 투자했으며, 앤스로픽은
구글과 클라우드 서비스 사용 계약도 체결
n 3대 클라우드 사업자인 구글, 마이크로소프트, 아마존은 차세대 AI 모델의 대표 기업인
앤스로픽 및 오픈AI와 협력을 확대하는 추세
£구글, 앤스로픽에 최대 20억 달러 투자 합의 및 클라우드 서비스 제공
n 구글이 2023년 10월 27일 앤스로픽에 최대 20억 달러를 투자하기로 합의했으며, 이 중 5억
달러를 우선 투자하고 향후 15억 달러를 추가로 투자할 방침
∙ 구글은 2023년 2월 앤스로픽에 이미 5억 5,000만 달러를 투자한 바 있으며, 아마존도 지난 9월
앤스로픽에 최대 40억 달러의 투자 계획을 공개
∙ 한편, 2023년 11월 8일 블룸버그 보도에 따르면 앤스로픽은 구글의 클라우드 서비스 사용을 위해
4년간 30억 달러 규모의 계약을 체결
∙ 오픈AI 창업자 그룹의 일원이었던 다리오(Dario Amodei)와 다니엘라 아모데이(Daniela Amodei)
남매가 2021년 설립한 앤스로픽은 챗GPT의 대항마 ‘클로드(Claude)’ LLM을 개발
n 아마존과 구글의 앤스로픽 투자에 앞서, 마이크로소프트는 차세대 AI 모델의 대표 주자인 오픈
AI와 협력을 확대
∙ 마이크로소프트는 오픈AI에 앞서 투자한 30억 달러에 더해 2023년 1월 추가로 100억 달러를
투자하기로 하면서 오픈AI의 지분 49%를 확보했으며, 오픈AI는 마이크로소프트의 애저(Azure)
클라우드 플랫폼을 사용해 AI 모델을 훈련
£구글, 클라우드 경쟁력 강화를 위해 생성 AI 투자 확대
n 구글은 수익률이 높은

In [31]:
# 8. Agent Parser 정의 및 Stream으로 실행하기
from langchain_teddynote.messages import AgentStreamParser

agent_stream_parser = AgentStreamParser()

response_stream = agent_with_chat_history.stream(
    {"input": "구글이 앤스로픽에 투자한 금액을 문서에서 찾아줘"},
    config={"configurable": {"session_id": "abcd1234"}}
)

for step in response_stream:
    agent_stream_parser.process_agent_steps(step)



> Entering new None chain...
[도구 호출]
Tool: pdf_search
query: 구글 앤스로픽 투자 금액
Log: 
Invoking: `pdf_search` with `{'query': '구글 앤스로픽 투자 금액'}`




Invoking: `pdf_search` with `{'query': '구글 앤스로픽 투자 금액'}`


1. 정책/법제 2. 기업/산업 3. 기술/연구 4. 인력/교육
구글, 앤스로픽에 20억 달러 투자로 생성 AI 협력 강화
KEY Contents
n 구글이 앤스로픽에 최대 20억 달러 투자에 합의하고 5억 달러를 우선 투자했으며, 앤스로픽은
구글과 클라우드 서비스 사용 계약도 체결
n 3대 클라우드 사업자인 구글, 마이크로소프트, 아마존은 차세대 AI 모델의 대표 기업인
앤스로픽 및 오픈AI와 협력을 확대하는 추세
£구글, 앤스로픽에 최대 20억 달러 투자 합의 및 클라우드 서비스 제공
n 구글이 2023년 10월 27일 앤스로픽에 최대 20억 달러를 투자하기로 합의했으며, 이 중 5억
달러를 우선 투자하고 향후 15억 달러를 추가로 투자할 방침
∙ 구글은 2023년 2월 앤스로픽에 이미 5억 5,000만 달러를 투자한 바 있으며, 아마존도 지난 9월
앤스로픽에 최대 40억 달러의 투자 계획을 공개
∙ 한편, 2023년 11월 8일 블룸버그 보도에 따르면 앤스로픽은 구글의 클라우드 서비스 사용을 위해
4년간 30억 달러 규모의 계약을 체결
∙ 오픈AI 창업자 그룹의 일원이었던 다리오(Dario Amodei)와 다니엘라 아모데이(Daniela Amodei)
남매가 2021년 설립한 앤스로픽은 챗GPT의 대항마 ‘클로드(Claude)’ LLM을 개발
n 아마존과 구글의 앤스로픽 투자에 앞서, 마이크로소프트는 차세대 AI 모델의 대표 주자인 오픈
AI와 협력을 확대
∙ 마이크로소프트는 오픈AI에 앞서 투자한 30억 달러에 더해 2023년 1월 추가로 100억 달러를
투자하기로 하면서 오픈AI의 지분 49%를

In [32]:
########## 8. 에이전트를 실행하고 결과를 확인합니다. ##########

# 질의에 대한 답변을 출력합니다.
response = agent_with_chat_history.stream(
    {"input": "이전의 답변을 영어로 번역해 주세요"},
    # 세션 ID를 설정합니다.
    # 여기서는 간단한 메모리 내 ChatMessageHistory를 사용하기 때문에 실제로 사용되지 않습니다
    config={"configurable": {"session_id": "abc1d234"}},
)

for step in response:
    agent_stream_parser.process_agent_steps(step)



> Entering new None chain...
Please translate the previous answer into English.

> Finished chain.
[최종 답변]
Please translate the previous answer into English.


In [33]:
########## 8. 에이전트를 실행하고 결과를 확인합니다. ##########

# 질의에 대한 답변을 출력합니다.
response = agent_with_chat_history.stream(
    {
        "input": "2024년 프로야구 플레이오프 진출 5개팀을 검색해서 알려주세요. 한글로 답변하세요"
    },
    # 세션 ID를 설정합니다.
    # 여기서는 간단한 메모리 내 ChatMessageHistory를 사용하기 때문에 실제로 사용되지 않습니다
    config={"configurable": {"session_id": "abc456"}},
)

for step in response:
    agent_stream_parser.process_agent_steps(step)



> Entering new None chain...
[도구 호출]
Tool: tavily_search_results_json
query: 2024년 프로야구 플레이오프 진출 팀
Log: 
Invoking: `tavily_search_results_json` with `{'query': '2024년 프로야구 플레이오프 진출 팀'}`




Invoking: `tavily_search_results_json` with `{'query': '2024년 프로야구 플레이오프 진출 팀'}`


[{'url': 'https://realjace.tistory.com/entry/2024-프로야구-포스트시즌-플레이오프-한국시리즈-일정-총정리', 'content': "2024 프로야구 포스트시즌 플레이오프 한국시리즈 일정 총정리 2024 프로야구 포스트시즌 플레이오프 한국시리즈 일정 총정리 2024 프로야구 포스트시즌 경기방식 2024 프로야구 포스트시즌 일정 2024 프로야구 포스트시즌 경기방식 와일드카드 결정전에서 승리한 팀과 정규시즌 3위 팀이 맞붙습니다. 준플레이오프에서 승리한 팀과 정규시즌 2위 팀이 경기를 펼치게 됩니다. 플레이오프 승리팀과 정규시즌 1위 팀이 7전 4선 승제로 경기를 펼치게 됩니다. 2024 프로야구 포스트시즌 진출팀 특히 4위부터 6위까지 승차가 거의 없기 때문에 마지막까지 흥미진진할 것으로 보이는데요, 각 팀이 144경기를 소화해야 하며 9월 28일까지 정규시즌이 마무리될 것으로 보입니다. 2024 프로야구 포스트시즌 일정 플레이오프 1차전 플레이오프 2차전 플레이오프 3차전 플레이오프 4차전 '스포츠&게임 정보' 카테고리의 다른 글 2024 프로야구 포스트시즌 티켓 예매방법\xa0\xa0(3) 2024 프로야구 올스타전 투표결과 경기일정 티켓 예매방법 가격 총정리\xa0\xa0(0) 2024 프로야구 포스트시즌 티켓 예매방법 티스토리 홈 이동"}, {'url': 'https://m.blog.naver.com/hoga1201/2235711178

In [34]:
########## 8. 에이전트를 실행하고 결과를 확인합니다. ##########

# 질의에 대한 답변을 출력합니다.
response = agent_with_chat_history.stream(
    {"input": "이전의 답변을 SNS 게시글 형태로 100자 내외로 작성하세요."},
    # 세션 ID를 설정합니다.
    # 여기서는 간단한 메모리 내 ChatMessageHistory를 사용하기 때문에 실제로 사용되지 않습니다
    config={"configurable": {"session_id": "abc456"}},
)

for step in response:
    agent_stream_parser.process_agent_steps(step)



> Entering new None chain...
"새로운 정보가 궁금하다면, PDF 문서에서 직접 검색해보세요! 필요한 내용을 빠르게 찾아드릴게요. #정보검색 #PDF"

> Finished chain.
[최종 답변]
"새로운 정보가 궁금하다면, PDF 문서에서 직접 검색해보세요! 필요한 내용을 빠르게 찾아드릴게요. #정보검색 #PDF"


In [35]:
########## 8. 에이전트를 실행하고 결과를 확인합니다. ##########

# 질의에 대한 답변을 출력합니다.
response = agent_with_chat_history.stream(
    {"input": "이전의 답변에 한국 시리즈 일정을 추가하세요."},
    # 세션 ID를 설정합니다.
    # 여기서는 간단한 메모리 내 ChatMessageHistory를 사용하기 때문에 실제로 사용되지 않습니다
    config={"configurable": {"session_id": "abc456"}},
)

for step in response:
    agent_stream_parser.process_agent_steps(step)



> Entering new None chain...
[도구 호출]
Tool: tavily_search_results_json
query: 2023 한국 시리즈 일정
Log: 
Invoking: `tavily_search_results_json` with `{'query': '2023 한국 시리즈 일정'}`




Invoking: `tavily_search_results_json` with `{'query': '2023 한국 시리즈 일정'}`


[{'url': 'https://wonjuri.tistory.com/entry/2023-한국시리즈-일정-예매시간-예매방법-가격-총정리feat-LG-우승-보러가자', 'content': '2023 한국시리즈 예매방법 일정 좌석 총정리 . 2023년 프로야구 한국시리즈 일정. 이번 2023년 프로야구는 3위 ssg와 4위 nc의 준플레이오프부터 시작하여 2위 kt와 ssg,nc의 승자와 플레이오프 시리즈를 진행하며 마지막으로 lg와 플레이오프 우승팀과 마지막 한국 ...'}, {'url': 'https://m.blog.naver.com/yzzzii/223256323435', 'content': '11월 7일 (화) 오후 6시 30분부터 잠실에서 시작하는 KT와 LG의 2023 신한은행 SOL KBO 한국시리즈 입장권 예매를 내일 (6일) 오후 2시부터 시작합니다! 한국시리즈 입장권은 전량 예매로 진행될 예정이며, 취소분 발생시 경기 당일 2시간 전부터 해당 구장에서 현장 ...'}, {'url': 'https://memo301.tistory.com/561', 'content': '2023 프로야구 한국시리즈| 챔피언을 향한 열전, 일정 & 결과 총정리 | 프로야구, 포스트시즌, 한국시리즈 뜨거웠던 2023 프로야구 정규시즌이 끝나고, 이제 챔피언을 가리는 한국시리즈가 시작되었습니다. 긴 여정 끝에 정상에 오르기 위한 치열한 승부가 펼쳐지는 가운데, 팬들의 열기 또한 뜨겁습니다.'}, {'url': 'https://id